In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql.window import Window
import pyodbc

conf = SparkConf() \
    .set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .setMaster("local[*]") \
    .setAppName("ReadMongoDB")


sc = SparkContext(conf=conf)
sqlC = SQLContext(sc)

mongo_ip = "mongodb://localhost:27017/Nettruyendb_tho."
truyen = sqlC.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "Truyens_tho").load()


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
truyen.show(5)

+--------+--------------------+----------+-------------+---------+--------------+---------+-------------+--------------------+--------------------+--------------------+--------------+
|Xep_hang|                 _id|  cap_nhat|luot_danh_gia| luot_xem|nguoi_theo_doi|so_chuong|      tac_gia|            the_loai|             tieu_de|             tom_tat|    trang_thai|
+--------+--------------------+----------+-------------+---------+--------------+---------+-------------+--------------------+--------------------+--------------------+--------------+
|     4.5|{6706e41f98066a57...|2024-09-24|          543|   15.801|         4.506|       54|Đang cập nhật|Action, Manhwa, T...|Mạt Thế Zombie 82-08|Giới thiệu về Web...|Đang tiến hành|
|     4.5|{6706e42498066a57...|2024-10-08|           25|         |           215|       13|Đang cập nhật|Manhua, Ngôn Tình...| Ánh Trăng Của Cô Ấy|Giới thiệu về Web...|Đang tiến hành|
|     4.5|{6706e42698066a57...|2024-10-08|      2007300|4.277.557|       929.041

In [3]:
kafka_bootstrap_servers = "localhost:9092"
topic_truyen = "Datatopic" 
def write_to_kafka(df, topic):
    df.show(5)
    columns_to_fill = ['Xep_hang', 'cap_nhat', 'luot_danh_gia', 'luot_xem', 'nguoi_theo_doi', 'so_chuong', 'trang_thai']
    fill_values = {
        'Xep_hang': 'N/A',
        'cap_nhat': 'Chưa cập nhật',
        'luot_danh_gia': 0,
        'luot_xem': 0,
        'nguoi_theo_doi': 0,
        'so_chuong': 0,
        'trang_thai': 'Chưa cập nhật'
    }
    df = df.fillna({col: fill_values.get(col, None) for col in columns_to_fill if col in df.columns})
    df.show(5)
    df.printSchema()
    if "_id" not in df.columns:
        df = df.withColumn("_id", F.monotonically_increasing_id())
    df = df.withColumn("_id", F.col("_id").cast("string"))
    df = df.selectExpr("CAST(_id AS STRING) AS key", "to_json(struct(*)) AS value")

    df.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("topic", topic) \
        .option("checkpointLocation", "/tmp/kafka_checkpoint") \
        .save()

write_to_kafka(truyen, topic_truyen)


+--------+--------------------+----------+-------------+---------+--------------+---------+-------------+--------------------+--------------------+--------------------+--------------+
|Xep_hang|                 _id|  cap_nhat|luot_danh_gia| luot_xem|nguoi_theo_doi|so_chuong|      tac_gia|            the_loai|             tieu_de|             tom_tat|    trang_thai|
+--------+--------------------+----------+-------------+---------+--------------+---------+-------------+--------------------+--------------------+--------------------+--------------+
|     4.5|{6706e41f98066a57...|2024-09-24|          543|   15.801|         4.506|       54|Đang cập nhật|Action, Manhwa, T...|Mạt Thế Zombie 82-08|Giới thiệu về Web...|Đang tiến hành|
|     4.5|{6706e42498066a57...|2024-10-08|           25|         |           215|       13|Đang cập nhật|Manhua, Ngôn Tình...| Ánh Trăng Của Cô Ấy|Giới thiệu về Web...|Đang tiến hành|
|     4.5|{6706e42698066a57...|2024-10-08|      2007300|4.277.557|       929.041